set up imports

In [1]:
import datetime
import os
import time
import requests
import json
import pandas as pd
from utility.testingAPIKeyValidity import testKeys

/Users/justice/Library/CloudStorage/OneDrive-VirginiaTech/Documents/School/EAGERVisualizations/env/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


initalizae helper methods

In [2]:
def removeSpaces(string):
    string = string.lower()
    return string


def convertDateToTimestamp(date, currYear):
    date = date.split(",")
    date = date[0].split(" ")
    month = date[0]
    monthNum = None
    day = date[1]
    if month == "Jan":
        monthNum = 1
    elif month == "Feb":
        monthNum = 2
    elif month == "Mar":
        monthNum = 3
    elif month == "Apr":
        monthNum = 4
    elif month == "May":
        monthNum = 5
    elif month == "Jun":
        monthNum = 6
    elif month == "Jul":
        monthNum = 7
    elif month == "Aug":
        monthNum = 8
    elif month == "Sep":
        monthNum = 9
    elif month == "Oct":
        monthNum = 10
    elif month == "Nov":
        monthNum = 11
    elif month == "Dec":
        monthNum = 12
    else:
        print("Error: month is not in the range of Jan-Dec")
        return -1

    try:
        day = int(day)
        # add checking to the range of day to make sure its valid for its month
        if month in "JanMarMayJulAugOctDec":
            if day > 31:
                print("Error: day is not in the range of 1-31")
                return -1
        elif month in "AprJunSepNov":
            if day > 30:
                print("Error: day is not in the range of 1-30")
                return -1
        elif month == "Feb":
            if day > 28:
                print("Error: day is not in the range of 1-28")
                return -1

    except:
        print("Error: day is not an int")
        return -1

    return int(datetime.datetime(currYear, monthNum, day).timestamp())


# make a method called save_data_to_json that takes a param called data and is a dctionary, this method so that if any of the values are none or NaN, skip over that entry
def save_data_to_json(data):
    # create a new dictionary that will be returned at the end of the method
    newDict = {}
    # loop through all the keys in the data dictionary
    for key in data:
        # if the value of the key is not none or NaN, add it to the newDict
        if data[key] == None or data[key] == "NaN":
            newDict[key] = "viewentry"
        else:
            newDict[key] = data[key]
    # save the newDict to the weather_data.json file
    with open("weather_data.json", "w") as outfile:
        json.dump(newDict, outfile, indent=4, sort_keys=True)


set up global variables

In [3]:
preTestapiKeys = [
    "5bf8cd2a704c0c7027c23cf77acb79cb",
    "a43e4fa8f4893126918b75676cd106e6",
    "f6f90fd57e86f923624b9cef75cb1d94",
    "baf1e9f73dc9484ae19490582c19099d",
    "dd903cc2648b50119e69dfcfd00c8a7f",
    "059af59e7405830b0365c9fa623335e6",
    "d630ce2596a01ed25c9c92efa16b02bb",
]
file_paths = [
            "2016.xlsx",
            "2017.xlsx",
            "2018.xlsx",
            "2019.xlsx",
            "2020.xlsx",
            "2021.xlsx",
            "2022.xlsx",
            "2023.xlsx",
        ]
file_paths.reverse()

apiKeys = testKeys(preTestapiKeys)

200
200
200
Error: api call failed on key baf1e9f73dc9484ae19490582c19099d
{'cod': 429, 'message': 'Your account is temporary blocked due to exceeding of requests limitation of your subscription type. Please choose the proper subscription https://openweathermap.org/price'}
Error: api call failed on key dd903cc2648b50119e69dfcfd00c8a7f
{'cod': 429, 'message': 'Your account is temporary blocked due to exceeding of requests limitation of your subscription type. Please choose the proper subscription https://openweathermap.org/price'}
Error: api call failed on key 059af59e7405830b0365c9fa623335e6
{'cod': 429, 'message': 'Your account is temporary blocked due to exceeding of requests limitation of your subscription type. Please choose the proper subscription https://openweathermap.org/price'}
Error: api call failed on key d630ce2596a01ed25c9c92efa16b02bb
{'cod': 429, 'message': 'Your account is temporary blocked due to exceeding of requests limitation of your subscription type. Please choose

Open up shelter data

In [4]:
with open("inputData/shelter_data.json") as json_file:
            shelter_data = json.load(json_file)

keys = shelter_data.keys()
newDict = {}
for shelter in shelter_data:
    newDict[removeSpaces(shelter)] = shelter_data.get(shelter)
print(f'shelter dict == {len(shelter_data)}')
print(f'new dict == {len(newDict)}')


shelter dict == 220
new dict == 220


Take in excel data

In [5]:
df = pd.DataFrame()
apiKey = apiKeys.pop()
print(apiKey)

for file_path in file_paths:
    relative_file_path = "inputData/" + file_path
    print("file path == " + relative_file_path)
    data = pd.read_excel(relative_file_path)
    data["year"] = file_path.split(".")[0]
    if file_path == "2016.xlsx":
        data.rename(columns={"Date": "date"}, inplace=True)
    
    
    # print(f'the date column for the year {file_path.split(".")[0]} == {data["date"]}')
    # print(f'the destination column for the year {file_path.split(".")[0]} == {data["Destination"]}')
    df = pd.concat([df, data])
print(df.dtypes)

f6f90fd57e86f923624b9cef75cb1d94
file path == inputData/2023.xlsx
file path == inputData/2022.xlsx
file path == inputData/2021.xlsx
file path == inputData/2020.xlsx
file path == inputData/2019.xlsx
file path == inputData/2018.xlsx
file path == inputData/2017.xlsx
file path == inputData/2016.xlsx
Hiker trail name       object
Hiker Journal Link     object
Journal Story          object
Start location         object
Destination            object
date                   object
Today Miles            object
Latitude               object
Longitude              object
State                  object
Total Shelters        float64
Person #              float64
year                   object
dtype: object


In [6]:
weather_data = json.load(open("weather_data.json"))
print(len(weather_data))
# print the count of all the rows in the df that unique hiker journal links
print(f'number of excel rows == {len(df)}')
print(df["Hiker Journal Link"].nunique())

33853
number of excel rows == 79427
78743


In [7]:
# get the count of all the entreis in the df by year that have unique Hiker Journal Links
print(df.groupby("year")["Hiker Journal Link"].nunique())

eligibleEntriesCount = [0 for _ in range(2016, 2024)]
print(len(eligibleEntriesCount))
allColumnNames = df.columns.to_numpy()
print(allColumnNames)
# rename the "Latitude" and "Longitude" columns to "latitude" and "longitude"
df.rename(columns={"Latitude": "latitude", "Longitude": "longitude"}, inplace=True)
# loop through all the rows in the df
for idx, row in df.iterrows():
    currYear = int(row["year"]) - 2016
    if pd.notnull(row["longitude"]) and pd.notnull(row["latitude"]):
        eligibleEntriesCount[currYear] += 1
print(len(eligibleEntriesCount))
print(eligibleEntriesCount)

year
2016    19667
2017    16131
2018    14380
2019    10169
2020     3516
2021     7537
2022     6727
2023     1300
Name: Hiker Journal Link, dtype: int64
8
['Hiker trail name' 'Hiker Journal Link' 'Journal Story' 'Start location'
 'Destination' 'date' 'Today Miles' 'Latitude' 'Longitude' 'State'
 'Total Shelters' 'Person #' 'year']
8
[3321, 2638, 1562, 1157, 417, 928, 6269, 767]


In [8]:
update_count = 0
for index, row in df.iterrows():    
    if row["Destination"] in newDict:
        
        currYear = int(row["year"]) - 2016
        df.loc[index, "longitude"] = newDict[row["Destination"]]["cordinates"][
            "longitude"
        ]
        df.loc[index, "latitude"] = newDict[row["Destination"]]["cordinates"][
            "latitude"
        ]
        update_count += 1
    
        eligibleEntriesCount[currYear] += 1
print(f'updated {update_count} rows')
print(eligibleEntriesCount)

updated 5176 rows
[3321, 2638, 3213, 2332, 882, 1869, 7092, 888]


In [9]:
from thefuzz import fuzz
from thefuzz import process
import pandas as pd
import json

exclude_words = [
    "shelter",
    "shelters",
    "SHELTER",
    "sheltered",
    "Shelter",
    "Shelters",
    "SHELTERED",
    "SHELTERS",
]

def custom_scorer(s1, s2):
    s1_tokens = [token for token in s1.split() if token not in exclude_words]
    s2_tokens = [token for token in s2.split() if token not in exclude_words]
    return fuzz.token_set_ratio(s1_tokens, s2_tokens)

totalFuzz = 0
totalFuzzCountAdded = 0
for idx, row in df.iterrows():
    destination = row["Destination"]
    # add logic its Hiker Journal Link" is in my_data, then check to see if the longitude row has information that isnt "" or is empty, if it doesnt, update it
    
    if (
        not destination
        or destination == "viewentry"
        or destination == ""
        or destination == "view entry"
        or type(destination) != str
    ):
        #print(f"bad destination name = {destination}")
        continue

    # use pandas to check to see if the longitude row has information that isnt "" or is empty
    if pd.notna(row["longitude"]) and row["longitude"] != "":
        #print(f"2 already had longitude and latitude info, lat = {row['latitude']}, lon = {row['longitude']}, at destination = {destination} ")
        continue

    # print(
    #     #f"destination = {destination}, longitude = {row['longitude']}, latitude = {row['latitude']}"
    # )
    totalFuzz += 1
    

    best_match, score = process.extractOne(
        destination, shelter_data.keys(), scorer=custom_scorer
    )
    #print(f"original destination = {destination}, best_match = {best_match}, score = {score}")
    # print(
    #     f"actual destination = {destination}, best_match = {best_match}, score = {score}"
    # )

    if score >= 87:
        # print("update!")
        # print(f"actual destination = {destination}, best_match = {best_match}, score = {score}")
        row["Latitude"] = shelter_data[best_match]["cordinates"]["latitude"]
        row["Longitude"] = shelter_data[best_match]["cordinates"]["longitude"]
        totalFuzzCountAdded += 1
print(f"total fuzz = {totalFuzz}, total fuzz count added = {totalFuzzCountAdded}")

update!
actual destination = silver bald shelter, best_match = Siler Bald Shelter, score = 95
update!
actual destination = jerry cabin sheltet, best_match = Jerry Cabin Shelter, score = 100
update!
actual destination = abington gap shelter, best_match = Abingdon Gap Shelter, score = 92
update!
actual destination = chatfield shelter , best_match = Chatfield Shelter, score = 100
update!
actual destination = jenkins sheltet, best_match = Jenkins Shelter, score = 100
update!
actual destination = u. s. hgwy 501, best_match = 501 Shelter, score = 100
update!
actual destination = u. s. hgwy. 501, best_match = 501 Shelter, score = 100
update!
actual destination = leaving punchbowl, best_match = Punchbowl Shelter, score = 100
update!
actual destination = steely-woodworth shelter, best_match = Seeley-Woodworth Shelter, score = 94
update!
actual destination = gravel springs, best_match = Gravel Springs Hut, score = 100
update!
actual destination = hawk mountain, best_match = Hawk Mountain Shelter

Applied processor reduces input query to empty string, all comparisons will have score 0. [Query: ' ']
Applied processor reduces input query to empty string, all comparisons will have score 0. [Query: ' ']


update!
actual destination = wayah bald nc , best_match = Wayah Bald Shelter, score = 100
update!
actual destination = rock gap nc, best_match = Rock Gap Shelter, score = 100
update!
actual destination = deep gap nc, best_match = Deep Gap Shelter, score = 100
update!
actual destination = rock gap nc, best_match = Rock Gap Shelter, score = 100
update!
actual destination = springer mountain ga, best_match = Springer Mountain, score = 100
update!
actual destination = laurel fork th side trail, best_match = Laurel Fork Shelter, score = 100
update!
actual destination = whitley shelter, best_match = Whitley Gap Shelter, score = 100
update!
actual destination = muskrat creek, best_match = Muskrat Creek Shelter, score = 100
update!
actual destination = pochuck shelter, best_match = Pochuck Mountain Shelter, score = 100
update!
actual destination = brown fork gap shelter , best_match = Brown Fork Gap Shelter, score = 100
update!
actual destination = flint mountain shelter , best_match = Flint M

In [10]:
# print the count of all the rows in the df that unique hiker journal links and have some number in the latitude and longitude columns
newCountArr = 0
seenLinks = set()
for _, rows in df.iterrows():
    if rows["Hiker Journal Link"] in seenLinks:
        continue
    if pd.notnull(rows["longitude"]) and pd.notnull(rows["latitude"]):
        newCountArr += 1
        seenLinks.add(rows["Hiker Journal Link"])
print(f"new count = {newCountArr}")

new count = 33936


In [11]:
# print the count of all the rows in the df that unique hiker journal links and have some number in lat and long split by the years
newCountArr = [0 for _ in range(2016, 2024)]
seenLinks = set()
for _, rows in df.iterrows():
    if rows["Hiker Journal Link"] in seenLinks:
        continue
    if pd.notnull(rows["longitude"]) and pd.notnull(rows["latitude"]) and rows["latitude"] != "" and rows["longitude"] != "":
        newCountArr[int(rows["year"]) - 2016] += 1
        seenLinks.add(rows["Hiker Journal Link"])
print(f"new count = {newCountArr}")

new count = [6882, 6191, 4516, 3893, 1802, 3315, 6312, 1025]
